In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
from math import sqrt
from scipy.stats import pearsonr, spearmanr

from env import get_connection
import wrangle_zillow
import prepare
import env
import acquire
import wrangle_mall


# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = acquire.get_zillow()
df.head()

,Unnamed: 0,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
df = df.drop(columns = ['Unnamed: 0'])

# Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,77380.0,1.495126e+06,8.609057e+05,3.490000e+02,7.520500e+05,1.497870e+06,2.240480e+06,2.982274e+06
parcelid,77380.0,1.300715e+07,3.481368e+06,1.071186e+07,1.153830e+07,1.253155e+07,1.421184e+07,1.676893e+08
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03
bathroomcnt,77380.0,2.299134e+00,9.966566e-01,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
bedroomcnt,77380.0,3.053489e+00,1.139103e+00,0.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.600000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
buildingqualitytypeid,49671.0,6.534638e+00,1.721933e+00,1.000000e+00,6.000000e+00,6.000000e+00,8.000000e+00,1.200000e+01
calculatedbathnbr,76771.0,2.316871e+00,9.797606e-01,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [6]:
df.dtypes

id                            int64
parcelid                      int64
airconditioningtypeid       float64
architecturalstyletypeid    float64
basementsqft                float64
                             ...   
buildingclassdesc            object
heatingorsystemdesc          object
propertylandusedesc          object
storydesc                    object
typeconstructiondesc         object
Length: 68, dtype: object

In [7]:
df.shape

(77380, 68)

In [8]:
def null_counts(df):
    
    new_columns = ['name', 'num_rows_missing', 'pct_rows_missing']
    
    new_df = pd.DataFrame(columns=new_columns)
    
    for col in list(df.columns):
        num_missing = df[col].isna().sum()
        pct_missing = (num_missing / df.shape[0]) * 100
        
        add_df = pd.DataFrame([{'name': col, 'num_rows_missing': num_missing,
                               'pct_rows_missing': pct_missing}])
        
        new_df = pd.concat([new_df, add_df], axis=0)
        
    new_df.set_index('name', inplace=True)
    
    return new_df

In [9]:
null_df = null_counts(df)

In [10]:
null_df.sort_values(by = 'pct_rows_missing', ascending = False)

,num_rows_missing,pct_rows_missing
name,,
buildingclassdesc,77365,99.980615
buildingclasstypeid,77365,99.980615
finishedsquarefeet13,77339,99.947015
storytypeid,77330,99.935384
basementsqft,77330,99.935384
...,...,...
latitude,0,0.000000
fips,0,0.000000
bedroomcnt,0,0.000000


In [11]:
df_minus_nulls = null_df[null_df['pct_rows_missing'] < .90]

In [12]:
df_minus_nulls.shape

(26, 2)

In [13]:
df_minus_nulls


,num_rows_missing,pct_rows_missing
name,,
id,0,0.000000
parcelid,0,0.000000
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
calculatedbathnbr,609,0.787025
calculatedfinishedsquarefeet,196,0.253295
fips,0,0.000000
fullbathcnt,609,0.787025
latitude,0,0.000000


In [14]:
df = df[df['bathroomcnt'] != 0]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76788 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            76788 non-null  int64  
 1   parcelid                      76788 non-null  int64  
 2   airconditioningtypeid         24936 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   76788 non-null  float64
 6   bedroomcnt                    76788 non-null  float64
 7   buildingclasstypeid           0 non-null      float64
 8   buildingqualitytypeid         49599 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    612 non-null    float64
 11  finishedfloor1squarefeet      6011 non-null   float64
 12  calculatedfinishedsquarefeet  76777 non-null  float64
 13  f

In [16]:
test_df = df

In [17]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    
    prop_null_column = 1 - prop_required_column
    
    for col in list(df.columns):
        
        null_sum = df[col].isna().sum()
        
        null_pct = null_sum / df.shape[0]
        
        if null_pct > prop_null_column:
            df.drop(columns=col, inplace=True)
            
    row_threshold = int(prop_required_row * df.shape[1])
    
    df.dropna(axis=0, thresh=row_threshold, inplace=True)
    
    return df
    

In [18]:
handle_missing_values(df, 0.6, 0.6)

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,NaN,Single Family Residential
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,NaN,Single Family Residential
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,NaN,Single Family Residential
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,-0.002245,2017-09-20,Central,Condominium
77376,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
77377,2968375,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,2.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,NaN,Single Family Residential
77378,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [19]:
mall_df = wrangle_mall.get_mall()

In [20]:
mall_df.head()

,Unnamed: 0,customer_id,gender,age,annual_income,spending_score
0,0,1,Male,19,15,39
1,1,2,Male,21,15,81
2,2,3,Female,20,16,6
3,3,4,Female,23,16,77
4,4,5,Female,31,17,40


In [21]:
mall_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      200 non-null    int64 
 1   customer_id     200 non-null    int64 
 2   gender          200 non-null    object
 3   age             200 non-null    int64 
 4   annual_income   200 non-null    int64 
 5   spending_score  200 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 9.5+ KB


In [22]:
mall_df.describe()

,Unnamed: 0,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,99.500000,100.500000,38.850000,60.560000,50.200000
std,57.879185,57.879185,13.969007,26.264721,25.823522
min,0.000000,1.000000,18.000000,15.000000,1.000000
25%,49.750000,50.750000,28.750000,41.500000,34.750000
50%,99.500000,100.500000,36.000000,61.500000,50.000000
75%,149.250000,150.250000,49.000000,78.000000,73.000000
max,199.000000,200.000000,70.000000,137.000000,99.000000


In [23]:
mall_df.shape

(200, 6)

In [24]:
q1, q3 = mall_df['spending_score'].quantile([0.25, 0.75])
q1, q3

(34.75, 73.0)

In [25]:
ss_iqr = q3 - q1
ss_iqr

38.25

In [26]:
k = 1.5

ss_upper = q3 + k * ss_iqr

ss_lower = q1 - k * ss_iqr

print(ss_upper, "&",  ss_lower)

130.375 & -22.625


In [27]:
np.where(mall_df['spending_score'] > ss_upper, 1, 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [28]:
def upper_outlier_detector(dataframe, column, k = 1.5):
    
    q1, q3 = dataframe[column].quantile([0.25, 0.75])
    
    iqr = q3 - q1
    
    upper_bound = q3 + k * ss_iqr

    return np.where(dataframe[column] > upper_bound, 1, 0)

In [29]:
my_list = ['age', 'annual_income']

for col in my_list:
    mall_df[f'{col}_upper_outliers'] = upper_outlier_detector(mall_df, col)
mall_df.head()

,Unnamed: 0,customer_id,gender,age,annual_income,spending_score,age_upper_outliers,annual_income_upper_outliers
0,0,1,Male,19,15,39,0,0
1,1,2,Male,21,15,81,0,0
2,2,3,Female,20,16,6,0,0
3,3,4,Female,23,16,77,0,0
4,4,5,Female,31,17,40,0,0


In [30]:
mall_df[['age_upper_outliers', 'annual_income_upper_outliers']].sum()

age_upper_outliers              0
annual_income_upper_outliers    2
dtype: int64

In [31]:
mall_df[mall_df['annual_income_upper_outliers'] == 1]

,Unnamed: 0,customer_id,gender,age,annual_income,spending_score,age_upper_outliers,annual_income_upper_outliers
198,198,199,Male,32,137,18,0,1
199,199,200,Male,30,137,83,0,1


In [32]:
train, val, test = wrangle_mall.train_val_test(mall_df)

In [33]:
train.shape, val.shape, test.shape

((140, 8), (36, 8), (24, 8))

In [34]:
mall_df.dtypes

Unnamed: 0                       int64
customer_id                      int64
gender                          object
age                              int64
annual_income                    int64
spending_score                   int64
age_upper_outliers               int64
annual_income_upper_outliers     int64
dtype: object

In [35]:
mall_df = wrangle_mall.mall_dummies(mall_df)

In [36]:
mall_df.isna().value_counts()

customer_id  age    annual_income  spending_score  age_upper_outliers  annual_income_upper_outliers  Male 
False        False  False          False           False               False                         False    200
dtype: int64

In [37]:
train, val, test = wrangle_mall.scale_splits_mm(train, val, test)

In [38]:
train.head()

,Unnamed: 0,customer_id,gender,age,annual_income,spending_score,age_upper_outliers,annual_income_upper_outliers
169,169,170,Male,0.269231,0.590164,63,0,0
97,97,98,Female,0.173077,0.368852,50,0,0
31,31,32,Female,0.057692,0.122951,73,0,0
12,12,13,Female,0.769231,0.040984,15,0,0
35,35,36,Female,0.057692,0.147541,81,0,0


In [39]:
train = wrangle_mall.mall_dummies(train)
val = wrangle_mall.mall_dummies(val)
test = wrangle_mall.mall_dummies(test)